In [1]:
import os

files = os.listdir('.')

In [2]:
import pandas as pd

full_data = pd.read_csv('data/full_dataset.csv')


In [3]:
import re

results = {}

file_prefix = 'alpaca-tloen-alpaca-lora-7b'


for file in files:
    if file.startswith(file_prefix) and file.endswith('.txt'):
        print(file)
        with open(file, 'r') as f:
            
            data = f.readlines()
            if 'multi' in f:
                temp = [x.strip() for x in data]
                print(len(temp))
                start = -1
                fn = file.split('.')[0]
                results[fn] = []
                for i in range(len(temp)):
                    if (temp[i].startswith('Yes') or temp[i].startswith('No') or temp[i].startswith('Unclear')):
                        if start >= 0:
                            results[fn].append('\n'.join(temp[start:i]).strip())
                        start = i
    #                 else:
    #                     print(temp[i])
                results[fn].append('\n'.join(temp[start:i]))
            else:
                results[file.split('.')[0]] = ''.join(data).split('\n---------***----***-----------\n')
#                 results[file.split('.')[0]] = [x.strip() for x in f.readlines()]

alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_HR_nt.json.txt
alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_HR_sa.json.txt
alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_TOS_red.json.txt
alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_CC_319.json.txt
alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_HR_bc.json.txt
alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_TOS_twt.json.txt
alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_CC_318.json.txt
alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_TOS_tk.json.txt
alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_HR_ab.json.txt
alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_TOS_met.json.txt
alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_TOS_yt.json.txt


In [4]:
[len(x) for x in results.values()], len(full_data)

([773, 773, 773, 773, 773, 773, 773, 773, 773, 773, 773], 2204)

In [5]:
print([(x if 'only relative alive' in full_data['text'].iloc[x] else ' ') for x in range(len(full_data))])

[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',

In [6]:
import html

def clean_text(text):

    inp = re.sub(r'(u/.*? |@.*?( |$))', "<user> ", text)
    inp = re.sub(r'\u2019', "'", inp)
    inp = re.sub(r'[“”]', '"', inp)
    inp = re.sub(r'\\u[a-f0-9]{4}', "'", inp)
    inp = re.sub(r'\(https://.*\)', "(URL)", inp)

    inp = re.sub(r'[\s]{2,}', " ", inp)
    inp = html.unescape(inp).encode('ascii', errors='ignore').decode()
        
    return inp

full_data['text'] = [clean_text(x).strip() for x in full_data['text'].to_list()]

In [7]:
from collections import Counter
import re



predictions = {}
indices = []
indices_tokeep = []
hallucinations = 0
# print(full_data.iloc[0]['text'])
for k, v in results.items():
#     if len(full_data) < 773:
#         res = v[-len(full_data-1):]
#     else:
#         res = v

    res = v
    if len(indices) == 0:
        print(len(res))
        for i in range(len(res)):
            r = res[i]
            try:
#                 print(r + '^^^^^^^^^^^^^^^')
                inp = re.search(r'### Input:\n(Post:|)((.|\s)*?) Q:', r).group(2).strip()
                search = full_data[(full_data['text'] == inp)].index
                if len(search) < 1:
                    print(inp, full_data[full_data['text'] == inp].index)
                    indices.append([2107])
                indices.append(full_data[full_data['text'] == inp].index)
            except Exception as e:
                print(e, i)
                del v[i]
#                 indices.append([2107])
                
        indices = [x for y in indices for x in y]
        indices_tokeep = list(set(indices))
    else:
        print(len(res))
        for i in range(len(res)):
            r = res[i]
            try:
#                 print(r + '^^^^^^^^^^^^^^^')
                inp = re.search(r'### Input:\n(Post:|)((.|\s)*?) Q:', r).group(2).strip()
                search = full_data[(full_data['text'] == inp)].index
                if len(search) < 1:
                    print(inp, full_data[full_data['text'] == inp].index)
                    indices.append([2107])
                indices.append(full_data[full_data['text'] == inp].index)
            except Exception as e:
                print(e, i)
                del v[i]
            
#     print(k,res[1])
    predictions[k] = []
    for r in res:
        x = r.split('### Response:')[-1]
        coarse = (2 if x.strip().startswith('Yes') else (1 if x.strip().startswith('Unclear') else (0 if x.strip().startswith('No') else -1)))
        predictions[k].append(coarse)
#     print(Counter(predictions[k]))
    
    hallucinations += Counter(predictions[k])[-1]

773
his only relative alive. My father died two years ago in a car accident.  Since my grandad's lawyer gave us the news, the only subject discussed in our home was how lucky I was and how I should administer my inheritance. At first, I didn't really listen to all this since the lawyer told me I'd have to wait a few months until I got the money.  But a month ago, my stepdad approached me and started talking about the mortgage payments and the credit card debts and how nice of me it would be to help them out and share my inheritance with my family. I didn't like the conversation at all, and when I said that I'd prefer to spend it investing in stocks and getting a college degree, he didn't look happy.  The discussion seemed odd because my stepdad owns a repair shop and always brags about his income before friends. Mom is an assistant accountant. I'm not saying there couldn't be any financial troubles, but strangely I didn't hear them talking about money problems until now. Also, the spen

only relative alive. My father died two years ago in a car accident.  Since my grandad's lawyer gave us the news, the only subject discussed in our home was how lucky I was and how I should administer my inheritance. At first, I didn't really listen to all this since the lawyer told me I'd have to wait a few months until I got the money.  But a month ago, my stepdad approached me and started talking about the mortgage payments and the credit card debts and how nice of me it would be to help them out and share my inheritance with my family. I didn't like the conversation at all, and when I said that I'd prefer to spend it investing in stocks and getting a college degree, he didn't look happy.  The discussion seemed odd because my stepdad owns a repair shop and always brags about his income before friends. Mom is an assistant accountant. I'm not saying there couldn't be any financial troubles, but strangely I didn't hear them talking about money problems until now. Also, the spending on 

only relative alive. My father died two years ago in a car accident.  Since my grandad's lawyer gave us the news, the only subject discussed in our home was how lucky I was and how I should administer my inheritance. At first, I didn't really listen to all this since the lawyer told me I'd have to wait a few months until I got the money.  But a month ago, my stepdad approached me and started talking about the mortgage payments and the credit card debts and how nice of me it would be to help them out and share my inheritance with my family. I didn't like the conversation at all, and when I said that I'd prefer to spend it investing in stocks and getting a college degree, he didn't look happy.  The discussion seemed odd because my stepdad owns a repair shop and always brags about his income before friends. Mom is an assistant accountant. I'm not saying there couldn't be any financial troubles, but strangely I didn't hear them talking about money problems until now. Also, the spending on 

In [8]:



test_data = full_data.iloc[indices_tokeep]

len(full_data), len(indices_tokeep)

(2204, 772)

In [20]:
test_data.to_csv('test_final.csv')

In [21]:
full_data

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,original_id,label,textorig,prev_label,finegrained,comments,...,cc_319,target_group,target,notes,source,text_type,created_utc,subreddit,submission,reddit_id
0,0,0,0,Did he kill your sex life? Women are objects s...,1006.0,Violates,<p>Did he kill your sex life? Your sense of pr...,NaN,"['TOS_tk', 'HR_ab', 'HR_bc', 'CC_319', 'HR_nt'...",NaN,...,1.0,"['Sex/Gender identity or expression', 'Identit...",A group/attribute,"['Wilful promotion of hatred', 'degrades women...",edited,NaN,NaN,NaN,NaN,NaN
1,1,1,1,Don't support our public schools because child...,1018.0,Violates,<p>And dark money. Support our public schools...,NaN,"['HR_ab', 'HR_bc', 'HR_nt', 'TOS_met', 'TOS_tw...",NaN,...,0.0,"['Identity/beliefs', 'Age']",A group/attribute,"['This statement exposes children to hatred; ""...",edited,NaN,NaN,NaN,NaN,NaN
2,2,2,2,Tricky Dick. I hope you get hit by a car the n...,1022.0,Does not violate,<p>Tricky Dick.</p>,NaN,NaN,NaN,...,0.0,[],An individual,[],edited,NaN,NaN,NaN,NaN,NaN
3,3,3,3,Canadians are physically and mentally inferior...,1026.0,Violates,<p>If anyone out there in Lala Land doesn't be...,NaN,"['TOS_tk', 'HR_ab', 'HR_bc', 'CC_319', 'HR_nt'...",NaN,...,1.0,"['Identity/beliefs', 'National or ethnic origi...",A group/attribute,"['Wilful promotion of hatred', 'Statement asse...",edited,NaN,NaN,NaN,NaN,NaN
4,4,4,4,Is that cat waving a Nazi banner? People with ...,1031.0,Violates,<p>Is that cat waving a Nazi banner? Is its na...,NaN,"['TOS_tk', 'HR_ab', 'CC_318', 'HR_bc', 'CC_319...",NaN,...,1.0,"['Political belief/association', 'Identity/bel...",A group/attribute,"['Public incitement of hatred', 'encourages th...",edited,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2199,2199,2199,474,They feed you 3 meals a day. I learned after b...,NaN,Does not violate,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,reddit,comment,1.665692e+09,funny,y30zj0,t1_is76ttn
2200,2200,2200,2521,A scholarship is something that pays for a por...,NaN,Does not violate,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,reddit,comment,1.685665e+09,NoStupidQuestions,13xwq07,t1_jmjsc18
2201,2201,2201,98,The cookie tin sewing supply stash seems to tr...,NaN,Does not violate,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,reddit,comment,1.670387e+09,AmItheAsshole,zep8g8,t1_iz875uz
2202,2202,2202,2851,I think Sony is going to release a Home like p...,NaN,Does not violate,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,reddit,comment,1.665608e+09,funny,y2bw1p,t1_is2kcmd


In [22]:
hallucinations/sum([len(x) for x in predictions.values()])

0.0023551577955723034

In [23]:
[i for i in range(len(results['{0}_multitarget_nonotes'.format(file_prefix.strip('_'))])) if 'sore losers' in results['{0}_multitarget_nonotes'.format(file_prefix.strip('_'))][i]]

KeyError: 'alpaca-tloen-alpaca-lora-7b_multitarget_nonotes'

In [24]:
len(results['{0}_multitarget_nonotes'.format(file_prefix)])

KeyError: 'alpaca-tloen-alpaca-lora-7b_multitarget_nonotes'

In [25]:
len(full_data)

2204

In [26]:

legal_defs = pd.read_csv('legal definitions.csv')
legal_labels = legal_defs['name'].to_list()
defs = pd.read_csv('data/definitions.csv')

In [27]:
import numpy as np

from sklearn.metrics import f1_score, recall_score, precision_score, hamming_loss
fine = []
pred = []

scores = [[],[],[]]
for i in range(len(legal_labels)):
    for r in results.keys():
        if legal_labels[i] in r:
            finegrained_lab = [(1 if full_data.iloc[j]['label'] == 'Meaning unclear' 
                                else (2 if (not pd.isna(full_data.iloc[j]['finegrained']) and legal_labels[i].lower() in full_data.iloc[j]['finegrained'].lower()) 
                                 else 0)) 
                               for j in range(len(full_data))]
            
            
            
            fine.append([(1 if x > 1 else 0) for x in finegrained_lab])
            pred.append([(1 if x > 1 else 0) for x in predictions[r]])
            print(r)
            print(f1_score(finegrained_lab, predictions[r], average='micro'))
            print(f1_score(finegrained_lab, predictions[r], average='macro'))
            


alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_CC_318


ValueError: Found input variables with inconsistent numbers of samples: [2204, 772]

In [28]:
results.keys()

dict_keys(['alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_HR_nt', 'alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_HR_sa', 'alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_TOS_red', 'alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_CC_319', 'alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_HR_bc', 'alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_TOS_twt', 'alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_CC_318', 'alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_TOS_tk', 'alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_HR_ab', 'alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_TOS_met', 'alpaca-tloen-alpaca-lora-7b_singletarget_nonotes_TOS_yt'])

In [17]:
coarse = [[] for _ in range(len(full_data))]

mapping = {'Violates': 2, 'Meaning unclear': 1, 'Does not violate': 0}

coarse_lab = [mapping[x] for x in full_data['label'].to_list()]

for r in predictions.keys():
    name = '_'.join(r.split('_')[-2:]).split('.')[0]
    print(name)
    if name in legal_labels:
        print(len(predictions[r]))
        for i in range(len(predictions[r])):
            coarse[i].append(predictions[r][i])
            

            
coarse = [Counter(x) for x in coarse]

coarse_pred = [(1 if x[1] > 4 else sorted(x.items(), key=lambda x: x[1], reverse=True)[0][0]) for x in coarse]
coarse_pred

HR_nt
772
HR_sa
772
TOS_red
772
CC_319
772
HR_bc
772
TOS_twt
772
CC_318
772
TOS_tk
772
HR_ab
772
TOS_met
772
TOS_yt
772


IndexError: list index out of range

In [18]:
f1_score(coarse_lab, coarse_pred, average='macro'), precision_score(coarse_lab, coarse_pred, average='macro'), recall_score(coarse_lab, coarse_pred, average='macro'), f1_score(coarse_lab, coarse_pred, average='micro')

NameError: name 'coarse_pred' is not defined

In [19]:
fine = np.transpose(np.array(fine))
pred = np.transpose(np.array(pred))

In [ ]:
f1_score(fine, pred, average='macro'), precision_score(fine, pred, average='macro'), recall_score(fine, pred, average='macro'), f1_score(fine, pred, average='micro')

In [ ]:
results.keys()

In [ ]:
import tiktoken

enc = tiktoken.encoding_for_model("gpt-4")

len(enc.encode(''.join(full_data['text'].to_list())))

In [ ]:
pd.Series([len(enc.encode(x)) for x in full_data['text'].to_list()]).describe()

In [ ]:
import json

with open('falcon_answers.json', 'r') as f:
    results = json.loads(f.readline())